In [16]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import calendar
from scipy import sparse
from pandas.api.types import CategoricalDtype
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import time
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
cd D:\Course\SBRS

D:\Course\SBRS


In [3]:
# ctg_file = 'datasets/retailrocket/category_tree.csv'
events_file = 'datasets/retailrocket/events.csv'
# item_prop_1 = 'datasets/retailrocket/item_properties_part1.csv'
# item_prop_2 = 'datasets/retailrocket/item_properties_part2.csv'

In [4]:
# ctg_df = pd.read_csv(ctg_file)
events_df = pd.read_csv(events_file)
# item_prop_1_df = pd.read_csv(item_prop_1)
# item_prop_2_df = pd.read_csv(item_prop_2)


In [5]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
events_actions = {
    'view': 1,
    'addtocart':2,
    'transaction':3
}

In [7]:
events_df.event = [events_actions[item] for item in events_df.event]

In [8]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,1,355908,NaN
1,1433224214164,992329,1,248676,NaN
2,1433221999827,111016,1,318965,NaN
3,1433221955914,483717,1,253185,NaN
4,1433221337106,951259,1,367447,NaN


In [9]:
events_df['event_date'] = pd.to_datetime(events_df['timestamp'])
events_df = events_df.drop('timestamp', axis=1)


In [10]:
events_df['date'] = events_df.event_date.dt.date
events_df['time'] = events_df.event_date.dt.time
events_df['year'] = events_df.event_date.dt.year
events_df['month'] = events_df.event_date.dt.month
events_df['day'] = events_df.event_date.dt.day
events_df['hour'] = events_df.event_date.dt.hour
events_df['minute'] = events_df.event_date.dt.minute
events_df['sec'] = events_df.event_date.dt.second
events_df = events_df.drop(['event_date','time','date'], axis=1)

In [11]:
events_df.head(1000000)

,visitorid,event,itemid,transactionid,year,month,day,hour,minute,sec
0,257597,1,355908,NaN,1970,1,1,0,23,53
1,992329,1,248676,NaN,1970,1,1,0,23,53
2,111016,1,318965,NaN,1970,1,1,0,23,53
3,483717,1,253185,NaN,1970,1,1,0,23,53
4,951259,1,367447,NaN,1970,1,1,0,23,53
...,...,...,...,...,...,...,...,...,...,...
999995,859313,1,452908,NaN,1970,1,1,0,24,0
999996,6400,1,342021,NaN,1970,1,1,0,24,0
999997,643318,1,322887,NaN,1970,1,1,0,24,0
999998,542030,3,314824,4569.0,1970,1,1,0,24,0


In [12]:
new = events_df.sort_values('visitorid')
new.head(10000)

,visitorid,event,itemid,transactionid,year,month,day,hour,minute,sec
1361687,0,1,285930,NaN,1970,1,1,0,24,2
1367212,0,1,357564,NaN,1970,1,1,0,24,2
1367342,0,1,67045,NaN,1970,1,1,0,24,2
830385,1,1,72028,NaN,1970,1,1,0,23,59
735202,2,1,342816,NaN,1970,1,1,0,23,58
...,...,...,...,...,...,...,...,...,...,...
1934064,5213,1,309502,NaN,1970,1,1,0,23,52
1937074,5213,1,412134,NaN,1970,1,1,0,23,52
1936948,5213,1,359310,NaN,1970,1,1,0,23,52
1927901,5213,1,237400,NaN,1970,1,1,0,23,52


In [14]:
list_df = []
for i in tqdm(range(len(events_df.visitorid.unique())//100)):
    time.sleep(0.00000001)
    list_df.append(events_df[events_df.visitorid == i].itemid.to_list())
list_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14075/14075 [03:38<00:00, 64.35it/s]


[[285930, 357564, 67045],
 [72028],
 [216305, 325215, 342816, 325215, 342816, 259884, 216305, 325215],
 [385090],
 [177677],
 [61396],
 [344723, 344723, 65273, 253615, 344723, 344723],
 [164941, 139394, 226353],
 [434230],
 [222422],
 [248766],
 [448220],
 [70225],
 [320589, 278689],
 [330269],
 [22495],
 [382645],
 [138434],
 [209302],
 [84663],
 [176623],
 [76091],
 [86411, 86411],
 [44608, 283916, 283916],
 [449369, 346051],
 [344325],
 [164342],
 [314714],
 [295343],
 [299118],
 [425758],
 [465565],
 [465465, 282491],
 [185254],
 [199885],
 [187946],
 [176286, 176286],
 [168952, 168952, 168952, 168952, 168952, 434344, 168952, 168952],
 [182101],
 [76740, 76740],
 [164625],
 [28961],
 [60399],
 [434782],
 [11729],
 [451993],
 [3780, 3780],
 [161925],
 [215123],
 [156173],
 [331168],
 [49967, 429304, 198762, 358388, 279059, 198762],
 [108504],
 [217218],
 [283115,
  283115,
  442228,
  442228,
  319680,
  388096,
  388096,
  319680,
  38965,
  249114,
  38965],
 [399556, 399556],
 [2

In [17]:
tr = TransactionEncoder()
tr_arr = tr.fit(list_df).transform(list_df)
df = pd.DataFrame(tr_arr, columns=tr.columns_)

In [19]:
df.head(100)

,6,90,112,147,199,263,391,403,449,460,...,466656,466657,466675,466735,466740,466747,466772,466847,466863,466864
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
96,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
97,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [33]:
frequent_itemsets = apriori(df, min_support = 0.0005, use_colnames = True)


In [34]:
frequent_itemsets

,support,itemsets
0,0.001066,(5411)
1,0.000639,(17114)
2,0.000853,(65273)
3,0.000639,(69533)
4,0.000568,(76426)
5,0.000568,(82389)
6,0.000924,(91755)
7,0.000924,(96924)
8,0.000782,(102306)
9,0.000782,(133814)


In [41]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.000005)

In [42]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [ ]:
events_df.transactionid.fillna(-1, inplace=True)

In [ ]:
events_df.head()

In [ ]:
users = events_df.visitorid.unique()
items = events_df.itemid.unique()
shape = (len(users),len(items))

In [ ]:
# Categorical coding
users_cat = CategoricalDtype(categories=sorted(users), ordered=True)
items_cat = CategoricalDtype(categories=sorted(items), ordered=True)
users_index = events_df.visitorid.astype(users_cat).cat.codes
items_index = events_df.itemid.astype(items_cat).cat.codes

In [ ]:
# To csr
coo = sparse.coo_matrix((events_df.event, (users_index, items_index)), shape=shape)
csr = coo.tocsr()

In [ ]:
# Все записи (относящиеся к одному пользоветелю), временной промежуток между которыми менее 10 минут, - сессия 
